In [ ]:
# Import libraries
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Set display options for better readability in output
#pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", '{:,.2f}'.format)

## **Data Overview**

### Load and explore all datasets

➤ Load all raw CSV files into individual DataFrames and store them in a dictionary for easier handling and quick access during exploration.

In [ ]:
# Load all CSV files from the Brazilian E-Commerce dataset into separate DataFrames
data_path = "../data/brazilian/raw/"
files = os.listdir(data_path)

customers_raw = pd.read_csv(data_path + "olist_customers_dataset.csv")
geolocation_raw = pd.read_csv(data_path + "olist_geolocation_dataset.csv")
orders_raw = pd.read_csv(data_path + "olist_orders_dataset.csv")
items_raw = pd.read_csv(data_path + "olist_order_items_dataset.csv")
payments_raw = pd.read_csv(data_path + "olist_order_payments_dataset.csv")
reviews_raw = pd.read_csv(data_path + "olist_order_reviews_dataset.csv")
products_raw = pd.read_csv(data_path + "olist_products_dataset.csv")
sellers_raw = pd.read_csv(data_path + "olist_sellers_dataset.csv")
translation_raw = pd.read_csv(data_path + "product_category_name_translation.csv")

# Store all DataFrames in a dictionary for easier looping and inspection
dataframes_raw = {
    "customers": customers_raw,
    "geolocation": geolocation_raw,
    "orders": orders_raw,
    "items": items_raw,
    "payments": payments_raw,
    "reviews": reviews_raw,
    "products": products_raw,
    "sellers": sellers_raw,
    "translation": translation_raw,
}


The following tables are included in the Brazilian E-Commerce dataset:

- `customers`: customer information  
- `geolocation`: geographical coordinates by zip code prefix  
- `orders`: order details including status and timestamps  
- `items`: product-level details for each order  
- `payments`: payment methods, amounts and installment information 
- `reviews`: customer reviews and ratings  
- `products`: product attributes including category and dimensions
- `sellers`: seller information  
- `translation`: Portuguese-to-English product category mapping 

*Note: Original file names such as `olist_customers_dataset.csv` were renamed to simpler identifiers like `customers` for ease of use.*

➤  Summary of all tables using `.shape`, column names, and duplicate counts.

In [ ]:
# Define a function to summarize a dictionary of DataFrames
def summarize_dataframes(df_dict=dataframes_raw):
    """
    Takes a dictionary of DataFrames and returns a summary DataFrame
        with the following information for each:
        - name: the key name from the dictionary
        - rows: number of rows in the DataFrame
        - columns: number of columns
        - column_names: a list of column names
        - duplicates: number of duplicated rows
    """
    summary = []

    # Loop over each DataFrame in the dictionary
    for name, df in df_dict.items():
        summary.append(
            {
                "name": name,
                "rows": df.shape[0],
                "columns": df.shape[1],
                "column_names": list(df.columns),
                "duplicates": df.duplicated().sum(),
            }
        )
    # Return a summary DataFrame
    return pd.DataFrame(summary)


# Call the function and display the summary of all loaded DataFrames
summarize_dataframes(dataframes_raw)

➤  Quick sampling of 5 rows from each table for visual inspection.

In [ ]:
# Display a random sample of 5 rows from each DataFrame for a quick visual inspection
for name, df in dataframes_raw.items():
    print(f'{name.capitalize()}:')
    display(df.sample(5))
    print("-"*130)

➤ Column-wise overview including dtypes, missing values, and unique counts.

In [ ]:
# Quick overview of column properties (dtypes, missing values, uniques) for all DataFrames
def overview(df_dict=dataframes_raw):
    """
    Creates and displays a column-wise overview for each DataFrame in a dictionary.

    Parameters:
        df_dict (dict): A dictionary of DataFrames (e.g., {'orders': orders, ...})

    Displays:
        For each DataFrame:
            - Data type
            - Non-null count
            - Missing value count and percentage
            - Missing value percentage
            - Number of unique values
            - Unique values
    """
    for name, df in df_dict.items():
        print(f'{name.capitalize()}:')
        summary = pd.DataFrame(
                {
                    "dtype": df.dtypes,
                    "total": df.count(),
                    "missing_n": df.isna().sum(),
                    "missing_%": df.isna().mean() * 100,
                    "uniques_n": df.nunique(),
                    "uniques": [df[col].unique() for col in df.columns],
                }
        )
        display(summary)   
        print("-"*130)

overview(dataframes_raw)


➤ Quick statistical overview of all numeric columns in each raw table to spot any unusual values or patterns.

In [ ]:
# Summarize basic statistics of all numeric columns for each DataFrame in the dictionary
def describe_numeric_columns(df_dict=dataframes_raw):
    """
    Displays a transposed summary of descriptive statistics (.describe().T)
    for all numeric columns in each DataFrame within the given dictionary.

    Parameters:
    df_dict (dict): A dictionary where keys are table names and values are pandas DataFrames.

    Notes:
    - If a DataFrame has no numeric columns, a message is printed instead.
    - The output includes a visual summary using display() for easier inspection in notebooks.
    """
    for name, df in df_dict.items():
        print(f"{name.capitalize()}:")
        numeric_df = df.select_dtypes(include="number")

        if numeric_df.empty:
            print("No numeric columns to describe.")
        else:
            display(numeric_df.describe().T)

        print("-" * 130)


describe_numeric_columns()

## **Data Cleaning**

➤ Copy raw DataFrames into a new working dictionary to preserve the original data before cleaning.

In [ ]:
# Create a new dictionary with copies of all raw DataFrames
def copy_raw_dataframes(raw_dict, exclude=None):
    """
    Creates copies of raw DataFrames to preserve the original data before any cleaning steps.

    Parameters:
        raw_dict (dict): Dictionary containing raw DataFrames.
        exclude (list): List of table names to exclude from copying.

    Returns:
        dict: A new dictionary with copies of the DataFrames.
    """
    exclude = exclude or []
    copy_dict = {}

    for name, df in raw_dict.items():
        if name in exclude:
            continue
        copy_dict[name] = df.copy()

    return copy_dict


dataframes = copy_raw_dataframes(dataframes_raw, exclude=["geolocation"])  # exclude 'geolocation', which is not used in the analysis

### Handling missing data

➤ Missing product-related information

In [ ]:
# Copy the 'products' DataFrame from the dictionary for further processing
products = dataframes["products"].copy()

# Display the number of missing values in each column of the 'products' DataFrame
products.isna().sum()

In [ ]:
# Count the number of rows where any important product-related column is missing
products_missing_cols = [
    "product_category_name",
    "product_name_lenght",
    "product_description_lenght",
    "product_photos_qty"
]

number_missing_all = products[products_missing_cols].isna().any(axis=1).sum()

# Print the number and percentage of affected rows
print(
    f"{number_missing_all} rows have missing values in key product-related columns "
    f"({number_missing_all / products.shape[0]:.1%} of the products table)."
)

In [ ]:
# Drop rows where all key product-related columns are missing (category, name length, description length, and photo count)
products.dropna(subset=products_missing_cols, how="all", inplace=True)

# Display the number of missing values remaining in each column
products.isna().sum()

Rows missing `product_category_name` (610 in total) are dropped, as this column is critical for analyzing product-level trends in customer satisfaction. Without it, meaningful grouping and interpretation are not possible.

These rows also lack values in other key descriptive columns — such as `product_name_lenght`, `product_description_lenght`, and `product_photos_qty` — which are relevant for understanding how product presentation may affect customer perception. Since these fields are simultaneously missing, and no reliable imputation method is available, we exclude these rows entirely.

Other missing fields, such as product dimensions and weight, are not directly relevant to our current analysis. However, to ensure the dataset remains reusable for future projects, we will impute the missing values for `product_weight_g`, `product_length_cm`, `product_height_cm`, and `product_width_cm` using median values from products in the same category and a similar price range.

In [ ]:
# Define the columns that describe the product's physical dimensions
size_cols = [
    "product_weight_g",
    "product_length_cm",
    "product_height_cm",
    "product_width_cm",
]

# View rows where at least one of the size columns has missing values
products[products[size_cols].isna().any(axis=1)]

There is now only one row in the dataset with missing values (`product_id` equal to `'09ff539a621711667c43eba6a3bd8466'`), and it happens to be missing all four size-related columns. This product belongs to the `'bebe'` category. Despite this being an isolated case, we will write the following code in a generalized way to ensure it can be reused for other projects or datasets with more missing values.

In [ ]:
# Merge median item price into the products table to enable price-based imputation
items = dataframes["items"].copy()
price_by_product = items.groupby("product_id")["price"].median().reset_index()
products_price = products.merge(price_by_product, on="product_id", how="left")

In [ ]:
# Create price bins (quartiles) within each product category, based on median price
# This helps identify similar products by both category and price level
products_price["price_bin"] = products_price.groupby("product_category_name")[
    "price"
].transform(lambda x: pd.qcut(x, q=4, duplicates="drop"))

In [ ]:
# Impute missing size values using the median for each (category, price_bin) group
for col in size_cols:
    # Calculate the group-specific median for the current column
    group_medians = products_price.groupby(["product_category_name", "price_bin"])[
        col
    ].transform("median")
    # Fill missing values in the current column using the group medians
    products_price[col] = products_price[col].fillna(group_medians)


In [ ]:
# Print the number of missing values remaining in each column for verification
# Note: 'price_bin' may have missing values if a product belongs to a category with only one product,
# since quartile-based binning (qcut) cannot be applied in such cases.
print(products_price.isna().sum())
# Verify that the one product with missing values was successfully imputed
products_price[products_price["product_id"] == '09ff539a621711667c43eba6a3bd8466']

In [ ]:
# Remove the helper columns and save the cleaned/imputed data back to 'products'
products = products_price.drop(columns=["price", "price_bin"])

➤ Missing English translation for product category names

Another issue identified during the data overview stage is that the `products` table contains two category names that are missing from the `translation` table. To ensure consistency in our analysis, we will merge the two tables to add English category names and manually fill in the missing translations.

In [ ]:
# Merge products with the translation table
translation = dataframes["translation"].copy()

products_translated = pd.merge(products, translation, how="left", on="product_category_name")

# Identify product categories that are missing English translations
categories_missing_translation = products_translated[
    products_translated["product_category_name_english"].isna()
]["product_category_name"].unique()

print(f"Categories missing English translation: {categories_missing_translation}")

The category name `'pc_gamer'` can remain unchanged.
`'portateis_cozinha_e_preparadores_de_alimentos'` refers to kitchen appliances and food preparation tools and will be translated as `'kitchen_appliances_and_food_processors'`.

In [ ]:
# Manual translation for missing categories
category_translation_dict = {
    "pc_gamer": "pc_gamer",
    "portateis_cozinha_e_preparadores_de_alimentos": "kitchen_appliances_and_food_processors",
}

# Fill in missing translations using the manual dictionary
products_translated["product_category_name_english"] = products_translated[
    "product_category_name_english"
].fillna(products_translated["product_category_name"].map(category_translation_dict))


Now that we have handled the missing values in the `products` table and added English translations for all product categories, we can update the table in the dictionary. The `translation` table is no longer needed and will be removed.  
Columns related to product dimensions, weight, and the original Portuguese category names will be dropped at a later stage.

In [ ]:
# Update the 'products' table in the dictionary with the translated version
dataframes["products"] = products_translated

# Remove the 'translation' table from the dictionary since it's no longer needed
del dataframes["translation"]

### Handling duplicates in `reviews` table

During the data overview stage, we found that neither `review_ids` nor `order_ids` are unique in the `reviews` table. However, for a consistent and reliable analysis of customer satisfaction, it is essential to ensure a one-to-one relationship between reviews and orders — meaning each `order_id` should be associated with exactly one `review_id`, and each `review_id` should correspond to only one `order_id`.
This prevents double-counting reviews and ensures that satisfaction scores are accurately linked to individual orders.

➤ Handling duplicates in `reviews_id` column

In [ ]:
# Copy the reviews and items tables
reviews = dataframes["reviews"].copy()
items = dataframes["items"].copy()

# Filter rows where the same review_id appears more than once
duplicate_reviews = reviews[reviews.duplicated("review_id", keep=False)].copy()

# Count how many times each review_id appears
review_id_counts = duplicate_reviews["review_id"].value_counts()

# Add the count back to the duplicate_reviews dataframe
duplicate_reviews["review_id_count"] = duplicate_reviews["review_id"].map(review_id_counts)

# Merge with items table to inspect what products are tied to these duplicated reviews
duplicate_reviews_items = pd.merge(duplicate_reviews, items, how="left", on="order_id")

# Report the number of duplicated review_id entries
print(
    f"Number of duplicated review_id entries: {duplicate_reviews.shape[0]} out of {reviews.shape[0]} "
    f"({duplicate_reviews.shape[0] / reviews.shape[0]:.1%} of the reviews dataset)."
)

# Sort by frequency of review_id, then review_id and order_id for easier inspection
duplicate_reviews_items_sorted = duplicate_reviews_items.sort_values(
    ["review_id_count", "review_id", "order_id"], ascending=[False, True, True]
)

# Display top rows with the most duplicated review_ids
duplicate_reviews_items_sorted.head(15)

The table above shows examples of `review_id`s that appear multiple times, each linked to a different `order_id`. These cases likely involve multi-item purchases — either of the same or different products — that should have been recorded under a single `order_id`, but were incorrectly split while sharing the same review.

Since this duplication affects only 1.6% of the reviews and introduces ambiguity into the mapping between `orders` and `reviews`, we consider it safer to remove these inconsistent entries from the dataset to preserve the integrity of the analysis.

In [ ]:
# Identify review_ids that appear more than once
duplicated_ids = reviews["review_id"][reviews["review_id"].duplicated(keep=False)]

# Remove rows with duplicated review_ids
reviews_no_duplicated_review_id = reviews[~reviews["review_id"].isin(duplicated_ids)]

# Print summary
print(
    f"Number of rows in updated reviews table: {reviews_no_duplicated_review_id.shape[0]}"
)
print(
    f"Number of unique review_ids: {reviews_no_duplicated_review_id['review_id'].nunique()}"
)


➤ Handling duplicates in `order_id` column

In [ ]:
# Filter rows where the same order_id appears more than once
duplicate_orders = reviews_no_duplicated_review_id[
    reviews_no_duplicated_review_id.duplicated("order_id", keep=False)
].copy()

# Count how many times each order_id appears
order_id_counts = duplicate_orders["order_id"].value_counts()

# Add the count as a new column for easier inspection
duplicate_orders["order_id_count"] = duplicate_orders["order_id"].map(order_id_counts)

# Merge with items table to inspect which products are tied to these duplicated orders
duplicate_orders_items = pd.merge(duplicate_orders, items, how="left", on="order_id")

# Report the number of duplicated order_id entries
print(
    f"Number of duplicated order_id entries: {duplicate_orders.shape[0]} out of {reviews_no_duplicated_review_id.shape[0]} "
    f"({duplicate_orders.shape[0] / reviews_no_duplicated_review_id.shape[0]:.1%} of the reviews dataset after removing review_id duplicates)."
)

# Sort by frequency of order_id, then by order_id and review_id
duplicate_orders_items_sorted = duplicate_orders_items.sort_values(
    ["order_id_count", "order_id", "review_id"], ascending=[False, True, True]
)

# Display top rows with the most duplicated order_ids
duplicate_orders_items_sorted.head(15)

In many cases, the same `order_id` is associated with multiple `review_id`s, which might differ in content or score. These appear to be edited versions of the same customer review, mistakenly recorded as separate entries.
To ensure that each order is represented by only one review, we keep only the latest available review per `order_id`, based on the `review_answer_timestamp` field, which offers the most precise timestamp available.

In [ ]:
# Convert review_answer_timestamp to datetime
reviews_no_duplicated_review_id.loc[:, "review_answer_timestamp"] = pd.to_datetime(
    reviews_no_duplicated_review_id["review_answer_timestamp"], errors="coerce"
)

In [ ]:
# Sort and keep the latest review per order_id
reviews_cleaned = reviews_no_duplicated_review_id.sort_values(
    "review_answer_timestamp"
).drop_duplicates(subset="order_id", keep="last")

In [ ]:
# Print total number of rows and number of unique review_ids and order_ids in cleaned table
print(f"Number of rows in cleaned reviews table: {reviews_cleaned.shape[0]}")
print("Number of unique review_ids and order_ids:")
print(reviews_cleaned[['review_id', "order_id"]].nunique())


Now that both `review_id` and `order_id` columns are unique, we can update the `dataframes` dictionary with the `reviews_cleaned` table.

In [ ]:
# Update the 'reviews' table in the dictionary with the cleaned version
dataframes["reviews"] = reviews_cleaned

### Saving cleaned dataframes as `csv.` files

In [ ]:
# Save cleaned dataframes as .csv files
save_path = (
    "c:\\Users\\Olesya\\Документы\\Projects\\stackfuelpp\\data\\brazilian\\cleaned"
)

for name, df in dataframes.items():
    file_path = os.path.join(save_path, f"{name}.csv")
    df.to_csv(file_path, index=False)